In [1]:
import pandas as pd
import numpy as np
import re, os, glob, calendar

path = './Discharge/'
# path = './Discharge/inconsistent/'
extension = 'txt'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
print(result)

['E.1_DischargeWY1980-2004.txt']


In [2]:
for file in result:

    fp = open(file, 'r')
    getYear = int(fp.readlines()[6].split('-')[1])
    fp.close()
    
    fp = open(file, 'r')
    content = fp.read()  
    
    data = re.split('Date',content)
    data_list = []
    for i in range(len(data)):
        if i!=0:
            data_list.append((data[i].split('Total'))[0])
    
    
    year = getYear

    dummyDF = pd.DataFrame()

    for k in range(len(data_list)):
        data_table = pd.DataFrame([x.split('\t') for x in data_list[k].split('\n')])

        data_table = data_table.iloc[0:-1,0:-2]
        data_table.columns = data_table.iloc[0] 
        data_table = data_table[1:]

        data_melted= pd.melt(data_table, id_vars=[''])
        abbr_to_num = {name: num for num, name in enumerate(calendar.month_abbr) if num}
        headers =  ["day", "month", "Discharge"]
        data_melted.columns = headers
        for i in  range(len(data_melted["month"])):
            data_melted["month"][i] = abbr_to_num[data_melted["month"][i]]

        data_melted['year'] = np.where(((data_melted["month"]==1) | (data_melted["month"]==2) | (data_melted["month"]==3)), year+1, year)
        data_melted['Date'] = data_melted['year'].astype(str) +"-"+ data_melted["month"].astype(str)+"-"+ data_melted['day'].astype(str) 
        data_melted = data_melted[['Date', 'Discharge']]

        dummyDF = dummyDF.append([data_melted])  
        filter = dummyDF["Discharge"] != ""
        dfNew = dummyDF[filter]

        year = year+1
        
        fp.close()

    name = os.path.splitext(file)[0]
    dfNew.to_csv('{}.csv'.format(name), index=False)